In [2]:
import SimpleITK as itk
import pydicom
import numpy as np
from PIL import Image
typenames = ['CYST', 'FNH', 'HCC', 'HEM', 'METS']
typeids = [0, 1, 2, 3, 4]
# 读取文件序列
def read_dicom_series(dir_name):
    reader=itk.ImageSeriesReader()
    dicom_series=reader.GetGDCMSeriesFileNames(dir_name)
    reader.SetFileNames(dicom_series)
    images=reader.Execute()
    image_array = itk.GetArrayFromImage(images)
    return image_array

# 读取单个DICOM文件
def read_dicom_file(file_name):
    header = pydicom.read_file(file_name)
    image = header.pixel_array
    image = header.RescaleSlope * image + header.RescaleIntercept
    return image

# 读取mhd文件
def read_mhd_image(file_path):
    header = itk.ReadImage(file_path)
    image = itk.GetArrayFromImage(header)
    return np.array(image)


# 保存mhd文件
def save_mhd_image(image, file_name):
    header = itk.GetImageFromArray(image)
    itk.WriteImage(header, file_name)
    
# 根据文件名返回期项名
def return_phasename(file_name):
    phasenames = ['NC', 'ART', 'PV']
    for phasename in phasenames:
        if file_name.find(phasename) != -1:
            return phasename
# 读取DICOM文件中包含的病例ID信息
def read_patientId(dicom_file_path):
    ds = pydicom.read_file(dicom_file_path)
    return ds.PatientID
# 返回病灶类型和ID的字典类型的数据 key是typename value是typeid
def return_type_nameid():
    res = {}
    res['CYST'] = 0
    res['FNH'] = 1
    res['HCC'] = 2
    res['HEM'] = 3
    res['METS'] = 4
    return res

# 返回病灶类型ID和名称的字典类型的数据 key是typeid value是typename
def return_type_idname():
    res = {}
    res[0] = 'CYST'
    res[1] = 'FNH'
    res[2] = 'HCC'
    res[3] = 'HEM'
    res[4] = 'METS'
    return res

# 根据病灶类型的ID返回类型的字符串
def return_typename_byid(typeid):
    idname_dict = return_type_idname()
    return idname_dict[typeid]

# 根据病灶类型的name返回id的字符串
def return_typeid_byname(typename):
    nameid_dict = return_type_nameid()
    return nameid_dict[typename]

# 填充图像
def fill_region(image):
    # image.show()
    from scipy import ndimage
    image = ndimage.binary_fill_holes(image).astype(np.uint8)
    return image


# 将一个矩阵保存为图片
def save_image(image_arr, save_path):
    image = Image.fromarray(np.asarray(image_arr, np.uint8))
    image.save(save_path)
    
def show_image(image):
    img = Image.fromarray(image)
    img.show()
    
def adjust_ww_wc(image):
    image[image < -70] = -70
    image[image > 180] = 180
    image += 70
    return image

# 放缩一片矩阵
def resize_image(image, size):
    image = Image.fromarray(np.asarray(image, np.float32))
    image = image.resize((size, size))
    return np.array(image)

In [8]:
# mhd_path = '/home/give/Documents/dataset/MedicalImage/MedicalImage/ROI_Whole/train/15442_2737174_2_0_4/ART_ROI_Expand.mhd'
# mhd_image = read_mhd_image(mhd_path)
# show_image(mhd_image)
# after_adjust = adjust_ww_wc(mhd_image)
# show_image(after_adjust)

In [9]:
# resized_image = resize_image(after_adjust, 128)
# show_image(resized_image)